# Convert the SWMM model values of DWF to the WEST model 

In [1]:

import SWMMToWESTConvert.aggregateNetwork as agN
import SWMMToWESTConvert.updateWESTfiles as uw


In [2]:
#MODEL points for checking the flow
LIMOILOU_COL= "RA_180174" #"DOM_941798" pipe
PASCAL_COL = "U89A" #"U004" pipe
STSACRA_COL = "R689587" #"UNI_602608" pipe
BEAU_COL = 'Pompe-Beauport'
NOEST_COL= "R016595" #'U003_Nord_Ouest' pipe
linkMeasurementFlow = [LIMOILOU_COL,BEAU_COL,STSACRA_COL,NOEST_COL,PASCAL_COL] #Change this to the nodes within the path

WTP_TANK = "RA_606859"
ST_SACR_LEAF_NODE= "R006831" #"R007637" previous assumption (and wrong)


# Get pipe sections from detailed model ------------------------

In [11]:
file = 'DWF2022.inp' 

#Links measurement are assumed to be discharging into the trunk path (not in it)
trunkModels, branchesModels = agN.aggregateAndModelNetwork(file, WTP_TANK, linkMeasurementFlow,ST_SACR_LEAF_NODE)

------------------------Getting the elements from the network ------------------------------------------------
There were 17 subcatchments with area 0
There were 0 DWFs with inflow type different than FLOW
There were 0 DFs with other constituents than FLOW
-------------------------------Obtaining and modelling the Trunk -------------------------------------------------
There are 22 connections to the path
1  relevant branches
15  connections to the path to be converted into catchments
4  points to break the path were found
Number of resulting sections of the path 5
----------------------------
Final number of pipe sections  5
Final number of catchments:  9
Final number of tanks in series: 27
-------------------------------Obtaining and converting the branches --------------------------------------------------
--------------------------Obtaining and modelling branch  U004 -------------------------------------------
There are 12 connections to the path
2  relevant branches
10  connection

In [15]:
trunkModels


[[{'PipeName': 'UNI_5277 - UNI_602608',
   'AreaTank': 202.10018333333332,
   'Volmax': 215.8304252821392,
   'k': 0.0001440241735152871,
   'tanksIndexes': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
  {'PipeName': 'UNI_602607 - UNI_18252',
   'AreaTank': 263.2939349999999,
   'Volmax': 167.8980693817985,
   'k': 0.0005989608867814975,
   'tanksIndexes': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]},
  {'PipeName': 'UNI_18251 - DOM_35983',
   'AreaTank': 542.1056816666666,
   'Volmax': 565.9341873625616,
   'k': 0.0006655926091548684,
   'tanksIndexes': [20, 21, 22, 23, 24, 25]},
  {'PipeName': 'U002_Pompe_IREU - DOM_1631422',
   'AreaTank': 22.737000000000002,
   'Volmax': 20.00856,
   'k': 1.6184019415743647e-05,
   'tanksIndexes': [26]},
  {'PipeName': 'DOM_1640876 - DOM_1631424',
   'AreaTank': 428.6331,
   'Volmax': 576.0828864,
   'k': 0.0002456577839313046,
   'tanksIndexes': [27]}],
 [{'CatchmentName': 'UNI_5277 - UNI_602608(Catch)[previous]',
   'EndNode': False,
   'AreaCatchment': 1477700.0,

# Start modifying WEST MODEL --------------------------

In [6]:
import SWMMToWESTConvert.SWMMtoWESTConstants as STW_C
from SWMMToWESTConvert import updateWESTfiles as uf

connProps = {}
connProps[STW_C.VEL_MIN_CONN] = 18
connProps[STW_C.VEL_CLASSES_CONN]=[23.6,47.2,65.8,141.5,283,565.9,800,943.2,1800,2829.6]

connectorsDict = {}
for b, indexes in zip(['Trunk','DOM_1631422'],[range(1,10),range(1,7)]):
    connectors= []
    for c in indexes:
        connectors.append(connProps)
    connectorsDict[b] = connectors


#classes to be assigned to the west blocks by model
modelClasses = {STW_C.SEWER_CLASS: 'psvdmSewerTank_Classplus_decay',
                STW_C.CATCH_CLASS: 'Catchment_NoRetention_IndirectStormwater',
                STW_C.CONN_CLASS: 'Kosim1ToPSVDM_curve_tracerflow_ClassplusViral',
                STW_C.COMB_CLASS: 'psvdmTwoCombiner'}


#files 
layoutFile = '01-Data/06-LimitTanks10/ProjectLimitTanks.Layout.xml'
layoutFileMod = '02-Output/06-LimitTanks10/ProjectLimitTanksMOD.Layout.xml'


In [7]:
uf.updateWESTLayoutFile(layoutFile, layoutFileMod, modelClasses, trunkModels.copy(), branchesModels.copy(), connectorsDict)

Number of submodels found  94
The number of sewers found were  48 , catchments  15 , connectors  15  and combiners  16
There are no more catchments.
The number of created links was  38
There are no more catchments.
The number of created links was  54
The last element in the network was  Two_combiner15
